In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib

In [2]:
dataset_name = r"dataset.csv"
data = pd.read_csv(dataset_name, sep=';', encoding='windows-1251') 

In [3]:
data = data.fillna(10)

In [4]:
data = data.drop([f'wr{i}' for i in range(4, 9)], axis=1) 
data = data.drop([f'wb{i}' for i in range(4, 10)], axis=1) 
data = data.drop([f'ww{i}' for i in range(4, 8)], axis=1) 

In [5]:
data.head()

,wr1,wr2,wr3,wblack1,wb1,wb2,wb3,ww1,ww2,ww3,wz1,wz2,wz3,wz4,pred
0,0.044504,0.060390,0.106884,0.168612,0.083340,0.093626,0.095258,0.095571,0.109521,0.114733,0.196243,0.689336,10.000000,10.0,0.7500
1,0.030080,0.056448,0.062209,0.131685,0.065734,0.109037,0.120100,-0.012136,0.009346,0.063076,0.463594,0.145493,10.000000,10.0,0.1500
2,0.063378,0.077901,0.089353,0.089108,0.004201,0.026121,0.026849,0.093377,0.098947,0.110464,0.206775,0.505973,10.000000,10.0,0.7500
3,0.084155,0.104521,0.128896,0.119260,0.052116,0.074195,0.087291,0.059821,0.094428,0.118629,0.299186,0.190774,0.165366,10.0,0.2850
4,0.060063,0.076161,0.104568,0.126860,0.047750,0.064816,0.067846,0.050691,0.091414,0.096486,0.198817,0.310254,0.166508,10.0,0.0855


In [6]:
Y = data.pred
X = data.drop('pred', axis=1) 

In [7]:
seed = 7
# split into 67% for train and 33% for test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=seed)

In [8]:
pipeline = make_pipeline(preprocessing.StandardScaler(), Ridge())

In [9]:
pipeline.get_params()

{'memory': None,
 'ridge': Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 'ridge__alpha': 1.0,
 'ridge__copy_X': True,
 'ridge__fit_intercept': True,
 'ridge__max_iter': None,
 'ridge__normalize': False,
 'ridge__random_state': None,
 'ridge__solver': 'auto',
 'ridge__tol': 0.001,
 'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'steps': [('standardscaler',
   StandardScaler(copy=True, with_mean=True, with_std=True)),
  ('ridge', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001))]}

In [10]:
hyperparameters = {
}

In [11]:
model = GridSearchCV(pipeline, param_grid=hyperparameters, cv=10)
 
# Fit and tune model
model.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('ridge', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params=None, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
y_pred = model.predict(X_test)
r2_score(np.array(y_test), np.array(y_pred))

0.16054843802200225

Теперь Keras

In [13]:
from keras.models import Sequential
from keras.layers import Dense

C:\Users\Dim\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
max_r2_score = 0
while True:
    # split into 67% for train and 33% for test
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=seed)
    # create model
    model = Sequential()
    model.add(Dense(64, input_dim=14, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
    # Fit the model
    model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=150, batch_size=10)

    y_pred = model.predict(X_test)
    r2_score_ = r2_score(np.array(y_test), np.array(y_pred))
    if r2_score_ > max_r2_score:
        model.save(f'my_model{r2_score_}.h5')  # creates a HDF5 file 'my_model.h5'
        max_r2_score = r2_score_

Train on 130 samples, validate on 65 samples
Epoch 1/150
130/130 [==============================] - 0s 3ms/step - loss: 0.1003 - mean_absolute_error: 0.2777 - val_loss: 0.0903 - val_mean_absolute_error: 0.2762
Epoch 2/150
130/130 [==============================] - 0s 224us/step - loss: 0.1001 - mean_absolute_error: 0.2946 - val_loss: 0.0999 - val_mean_absolute_error: 0.2951
Epoch 3/150
130/130 [==============================] - 0s 231us/step - loss: 0.1042 - mean_absolute_error: 0.2985 - val_loss: 0.0991 - val_mean_absolute_error: 0.2993
Epoch 4/150
130/130 [==============================] - 0s 255us/step - loss: 0.0984 - mean_absolute_error: 0.2862 - val_loss: 0.0941 - val_mean_absolute_error: 0.2898
Epoch 5/150
130/130 [==============================] - 0s 216us/step - loss: 0.0963 - mean_absolute_error: 0.2917 - val_loss: 0.0939 - val_mean_absolute_error: 0.2891
Epoch 6/150
130/130 [==============================] - 0s 285us/step - loss: 0.0999 - mean_absolute_error: 0.2921 - val_lo

130/130 [==============================] - 0s 162us/step - loss: 0.0684 - mean_absolute_error: 0.2137 - val_loss: 0.0759 - val_mean_absolute_error: 0.2393
Epoch 50/150
130/130 [==============================] - 0s 231us/step - loss: 0.0673 - mean_absolute_error: 0.2027 - val_loss: 0.0988 - val_mean_absolute_error: 0.2800
Epoch 51/150
130/130 [==============================] - 0s 208us/step - loss: 0.0686 - mean_absolute_error: 0.2138 - val_loss: 0.0752 - val_mean_absolute_error: 0.2352
Epoch 52/150
130/130 [==============================] - 0s 208us/step - loss: 0.0693 - mean_absolute_error: 0.2166 - val_loss: 0.0822 - val_mean_absolute_error: 0.2229
Epoch 53/150
130/130 [==============================] - 0s 216us/step - loss: 0.0707 - mean_absolute_error: 0.2049 - val_loss: 0.1176 - val_mean_absolute_error: 0.2966
Epoch 54/150
130/130 [==============================] - 0s 201us/step - loss: 0.0685 - mean_absolute_error: 0.2129 - val_loss: 0.0763 - val_mean_absolute_error: 0.2324
Epoch

Epoch 98/150
130/130 [==============================] - 0s 216us/step - loss: 0.0488 - mean_absolute_error: 0.1744 - val_loss: 0.0983 - val_mean_absolute_error: 0.2449
Epoch 99/150
130/130 [==============================] - 0s 193us/step - loss: 0.0471 - mean_absolute_error: 0.1633 - val_loss: 0.0994 - val_mean_absolute_error: 0.2475
Epoch 100/150
130/130 [==============================] - 0s 301us/step - loss: 0.0451 - mean_absolute_error: 0.1655 - val_loss: 0.0908 - val_mean_absolute_error: 0.2357
Epoch 101/150
130/130 [==============================] - 0s 347us/step - loss: 0.0546 - mean_absolute_error: 0.1757 - val_loss: 0.0770 - val_mean_absolute_error: 0.2115
Epoch 102/150
130/130 [==============================] - 0s 208us/step - loss: 0.0533 - mean_absolute_error: 0.1817 - val_loss: 0.0755 - val_mean_absolute_error: 0.2214
Epoch 103/150
130/130 [==============================] - 0s 193us/step - loss: 0.0607 - mean_absolute_error: 0.1894 - val_loss: 0.1142 - val_mean_absolute_er

130/130 [==============================] - 0s 170us/step - loss: 0.0435 - mean_absolute_error: 0.1628 - val_loss: 0.0800 - val_mean_absolute_error: 0.2187
Epoch 147/150
130/130 [==============================] - 0s 193us/step - loss: 0.0426 - mean_absolute_error: 0.1575 - val_loss: 0.1164 - val_mean_absolute_error: 0.2618
Epoch 148/150
130/130 [==============================] - 0s 201us/step - loss: 0.0495 - mean_absolute_error: 0.1663 - val_loss: 0.0832 - val_mean_absolute_error: 0.2233
Epoch 149/150
130/130 [==============================] - 0s 247us/step - loss: 0.0400 - mean_absolute_error: 0.1537 - val_loss: 0.0943 - val_mean_absolute_error: 0.2382
Epoch 150/150
130/130 [==============================] - 0s 231us/step - loss: 0.0426 - mean_absolute_error: 0.1598 - val_loss: 0.0837 - val_mean_absolute_error: 0.2213
Train on 130 samples, validate on 65 samples
Epoch 1/150
130/130 [==============================] - 0s 2ms/step - loss: 0.1141 - mean_absolute_error: 0.3012 - val_loss: 

130/130 [==============================] - 0s 216us/step - loss: 0.0646 - mean_absolute_error: 0.2117 - val_loss: 0.0776 - val_mean_absolute_error: 0.2447
Epoch 45/150
130/130 [==============================] - 0s 208us/step - loss: 0.0642 - mean_absolute_error: 0.2107 - val_loss: 0.0875 - val_mean_absolute_error: 0.2658
Epoch 46/150
130/130 [==============================] - 0s 193us/step - loss: 0.0760 - mean_absolute_error: 0.2180 - val_loss: 0.0890 - val_mean_absolute_error: 0.2224
Epoch 47/150
130/130 [==============================] - 0s 239us/step - loss: 0.0698 - mean_absolute_error: 0.2153 - val_loss: 0.0800 - val_mean_absolute_error: 0.2524
Epoch 48/150
130/130 [==============================] - 0s 239us/step - loss: 0.0642 - mean_absolute_error: 0.2096 - val_loss: 0.0803 - val_mean_absolute_error: 0.2452
Epoch 49/150
130/130 [==============================] - 0s 231us/step - loss: 0.0602 - mean_absolute_error: 0.2003 - val_loss: 0.0826 - val_mean_absolute_error: 0.2505
Epoch

...

130/130 [==============================] - 0s 470us/step - loss: 0.0979 - mean_absolute_error: 0.2910 - val_loss: 0.0894 - val_mean_absolute_error: 0.2741
Epoch 7/150
130/130 [==============================] - 0s 447us/step - loss: 0.0971 - mean_absolute_error: 0.2895 - val_loss: 0.0946 - val_mean_absolute_error: 0.2898
Epoch 8/150
130/130 [==============================] - 0s 405us/step - loss: 0.0945 - mean_absolute_error: 0.2866 - val_loss: 0.0895 - val_mean_absolute_error: 0.2760
Epoch 9/150
130/130 [==============================] - 0s 378us/step - loss: 0.0964 - mean_absolute_error: 0.2823 - val_loss: 0.0984 - val_mean_absolute_error: 0.2991
Epoch 10/150
130/130 [==============================] - 0s 397us/step - loss: 0.1020 - mean_absolute_error: 0.2955 - val_loss: 0.0885 - val_mean_absolute_error: 0.2571
Epoch 11/150
130/130 [==============================] - 0s 440us/step - loss: 0.1002 - mean_absolute_error: 0.2785 - val_loss: 0.1003 - val_mean_absolute_error: 0.3016
Epoch 12

Epoch 55/150
130/130 [==============================] - 0s 328us/step - loss: 0.0672 - mean_absolute_error: 0.2092 - val_loss: 0.0757 - val_mean_absolute_error: 0.2422
Epoch 56/150
130/130 [==============================] - 0s 351us/step - loss: 0.0631 - mean_absolute_error: 0.2055 - val_loss: 0.1048 - val_mean_absolute_error: 0.2822
Epoch 57/150
130/130 [==============================] - 0s 393us/step - loss: 0.0551 - mean_absolute_error: 0.1932 - val_loss: 0.0720 - val_mean_absolute_error: 0.2249
Epoch 58/150
130/130 [==============================] - 0s 347us/step - loss: 0.0565 - mean_absolute_error: 0.1882 - val_loss: 0.0735 - val_mean_absolute_error: 0.2275
Epoch 59/150
130/130 [==============================] - 0s 374us/step - loss: 0.0585 - mean_absolute_error: 0.1887 - val_loss: 0.1030 - val_mean_absolute_error: 0.2648
Epoch 60/150
130/130 [==============================] - 0s 401us/step - loss: 0.0543 - mean_absolute_error: 0.1838 - val_loss: 0.0784 - val_mean_absolute_error:

Epoch 104/150
130/130 [==============================] - 0s 629us/step - loss: 0.0463 - mean_absolute_error: 0.1681 - val_loss: 0.0838 - val_mean_absolute_error: 0.2312
Epoch 105/150
130/130 [==============================] - 0s 636us/step - loss: 0.0455 - mean_absolute_error: 0.1598 - val_loss: 0.0710 - val_mean_absolute_error: 0.2052
Epoch 106/150
130/130 [==============================] - 0s 467us/step - loss: 0.0488 - mean_absolute_error: 0.1704 - val_loss: 0.0731 - val_mean_absolute_error: 0.2194
Epoch 107/150
130/130 [==============================] - 0s 463us/step - loss: 0.0458 - mean_absolute_error: 0.1662 - val_loss: 0.0975 - val_mean_absolute_error: 0.2514
Epoch 108/150
130/130 [==============================] - 0s 436us/step - loss: 0.0467 - mean_absolute_error: 0.1635 - val_loss: 0.0824 - val_mean_absolute_error: 0.2284
Epoch 109/150
130/130 [==============================] - 0s 417us/step - loss: 0.0494 - mean_absolute_error: 0.1721 - val_loss: 0.0697 - val_mean_absolute_

130/130 [==============================] - 0s 424us/step - loss: 0.0978 - mean_absolute_error: 0.2877 - val_loss: 0.0989 - val_mean_absolute_error: 0.2973
Epoch 3/150
130/130 [==============================] - 0s 405us/step - loss: 0.1006 - mean_absolute_error: 0.2905 - val_loss: 0.0898 - val_mean_absolute_error: 0.2632
Epoch 4/150
130/130 [==============================] - 0s 443us/step - loss: 0.0987 - mean_absolute_error: 0.2895 - val_loss: 0.0939 - val_mean_absolute_error: 0.2883
Epoch 5/150
130/130 [==============================] - 0s 521us/step - loss: 0.1024 - mean_absolute_error: 0.2893 - val_loss: 0.1023 - val_mean_absolute_error: 0.3015
Epoch 6/150
130/130 [==============================] - 0s 412us/step - loss: 0.0972 - mean_absolute_error: 0.2922 - val_loss: 0.0893 - val_mean_absolute_error: 0.2721
Epoch 7/150
130/130 [==============================] - 0s 343us/step - loss: 0.0978 - mean_absolute_error: 0.2889 - val_loss: 0.0907 - val_mean_absolute_error: 0.2805
Epoch 8/15

KeyboardInterrupt: 

In [15]:
max_r2_score

0.1787714645158932

In [16]:
from sklearn.svm import SVR

In [17]:
pipeline = make_pipeline(preprocessing.StandardScaler(), SVR())

In [20]:
model = GridSearchCV(pipeline, param_grid=hyperparameters, cv=10)
 
# Fit and tune model
model.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svr', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
y_pred = model.predict(X_test)
r2_score(np.array(y_test), np.array(y_pred))

0.1014482674560383